## 第一部分：了解 nn.Module的基本操作

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [3]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [4]:
for name, param in model.named_parameters():
    print(f'{name}, {param.shape}')

conv1.weight, torch.Size([64, 3, 7, 7])
bn1.weight, torch.Size([64])
bn1.bias, torch.Size([64])
layer1.0.conv1.weight, torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight, torch.Size([64])
layer1.0.bn1.bias, torch.Size([64])
layer1.0.conv2.weight, torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight, torch.Size([64])
layer1.0.bn2.bias, torch.Size([64])
layer1.1.conv1.weight, torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight, torch.Size([64])
layer1.1.bn1.bias, torch.Size([64])
layer1.1.conv2.weight, torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight, torch.Size([64])
layer1.1.bn2.bias, torch.Size([64])
layer2.0.conv1.weight, torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight, torch.Size([128])
layer2.0.bn1.bias, torch.Size([128])
layer2.0.conv2.weight, torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight, torch.Size([128])
layer2.0.bn2.bias, torch.Size([128])
layer2.0.downsample.0.weight, torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight, torch.Size([128])
layer2.0.downsample.1.bias, torch.Size([128])
l

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [6]:
input_ = torch.randn(1, 3, 128, 128)

In [5]:
for param in model.parameters():
    param.requires_grad=False

In [7]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [8]:
input_ = torch.randn(10,12)
## 示範
Linear = nn.Linear(12,10)
BN = nn.BatchNorm1d(10)

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.sequential = nn.Sequential(nn.Linear(12,10), nn.BatchNorm1d(10))
        self.repeat_linear = nn.ModuleList([nn.Sequential(nn.Linear(10, 10), nn.BatchNorm1d(10)) for _ in range(10)])
        self.output = nn.Linear(10,3)

    def forward(self, x):
        x = self.sequential(x)
        for module in self.repeat_linear:
            x = module(x)
        x = self.output(x)
        return x

In [10]:
model = Model()

In [11]:
model

Model(
  (sequential): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Sequential(
      (0): Linear(in_features=10, out_fe

In [12]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [13]:
outupt

tensor([[ 0.4854,  0.1840,  0.0306],
        [-0.7732,  0.2507,  0.0859],
        [-0.5564, -0.1432,  0.1806],
        [ 0.6725,  0.0359, -0.1217],
        [ 0.3243, -0.5029, -0.1981],
        [ 1.5090, -0.0291, -0.3520],
        [-0.5110, -0.2988,  0.2223],
        [ 1.5871, -0.9768,  0.1228],
        [-0.7778, -0.0967, -0.1984],
        [ 0.6692,  0.0564, -0.0704]], grad_fn=<AddmmBackward>)